In [7]:
using JSON: parsefile
using WGLMakie
using Gen
using StaticArrays
using Printf

norm(v) = sqrt(sum(v.^2))
import Base.angle

struct Pose
    position::SVector{2,Float64}
    angle::Float64
    orientation::SVector{2,Float64} 
end

position(pose::Pose) = pose.position
angle(pose::Pose) = pose.angle
orientation(pose::Pose) = pose.orientation
Pose(position, angle::Float64) = Pose(position, rem2pi(angle, RoundNearest), SVector(cos(angle), sin(angle)))
Pose(position, orientation::AbstractVector{Float64}) = Pose(position, atan(orientation[2], orientation[1]))
Base.show(io::IO, p::Pose) = print(io, "Pose(", p.position, " ", p.angle, ")")

struct Control
    ds ::Float64
    dhd::Float64
end

step_along_pose(pose::Pose, step_size::Float64) = position(pose) .+ step_size .* orientation(pose)
rotate_pose(pose::Pose, Δθ::Float64) = Pose(position(pose), angle(pose) + Δθ)

function integrate_controls_unphysical(start_pose::Pose, robot_inputs::Vector{Control})
    path = Vector{Pose}(undef, length(robot_inputs.controls) + 1)
    path[1] = start_pose
    for t in 1:length(robot_inputs)
        position_new = step_along_pose(path[t], robot_inputs[t])
        angle_new = path[t].hd + robot_inputs[t].dhd
        path[t+1] = Pose(position_new, angle_new)
    end
    return path
end

# %% [markdown]
# This code has the problem that it is **unphysical**: the walls in no way constrain the robot motion.
#
# We employ the following simple physics: when the robot's forward step through a control comes into contact with a wall, that step is interrupted and the robot instead "bounces" a fixed distance from the point of contact in the normal direction to the wall.

# %%
# Return unique s, t such that p + s*u == q + t*v.
function solve_lines(p, u, q, v; PARALLEL_TOL=1.0e-10)
    det = u[1] * v[2] - u[2] * v[1]
    if abs(det) < PARALLEL_TOL
        return nothing, nothing
    else
        s = (v[1] * (p[2]-q[2]) - v[2] * (p[1]-q[1])) / det
        t = (u[2] * (q[1]-p[1]) - u[1] * (q[2]-p[2])) / det
        return s, t
    end
end

function distance(pose, segment)
    s, t = solve_lines(position(pose), orientation(pose), segment.p1, segment.dp)
    # Solving failed (including, by fiat, if pose is parallel to segment) iff isnothing(s).
    # Pose is oriented away from segment iff s < 0.
    # Point of intersection lies on segment (as opposed to the infinite line) iff 0 <= t <= 1.
    return (isnothing(s) || s < 0. || !(0. <= t <= 1.)) ? Inf : s
end

function physical_step(p1, p2, hd, world_inputs)
    step_pose = Pose(p1, p2 .- p1)
    (s, i) = findmin(w -> distance(step_pose, w), world_inputs.walls)
    if s > norm(p2 .- p1)
        # Step succeeds without contact with walls.
        return Pose(p2, hd)
    else
        dp = orientation(step_pose)
        contact_point = p1 .+ s .* dp
        unit_tangent = world_inputs.walls[i].dp / norm(world_inputs.walls[i].dp)
        unit_normal = SVector(-unit_tangent[2], unit_tangent[1])
        # Sign of 2D cross product determines orientation of bounce.
        if dp[1] * world_inputs.walls[i].dp[2] - dp[2] * world_inputs.walls[i].dp[1] < 0.
            unit_normal = -unit_normal
        end
        return Pose(contact_point + world_inputs.bounce * unit_normal, hd)
    end
end

function integrate_controls(robot_init::Pose, robot_inputs::Vector{Control}, world_inputs)
    path = Vector{Pose}(undef, length(robot_inputs) + 1)
    path[1] = robot_init
    for t in 1:length(robot_inputs)
        position_new = step_along_pose(path[t], robot_inputs[t].dhd)
        angle_new = angle(path[t]) + robot_inputs[t].dhd
        # Perform the physical correction
        path[t+1] = physical_step(position(path[t]), position_new, angle_new, world_inputs)
    end
    return path
end


integrate_controls (generic function with 1 method)

In [8]:
include("world.jl")

load_world (generic function with 1 method)

In [9]:
const World = @NamedTuple{walls::Vector{Segment}, clutters::Vector{Vector{Segment}}, walls_clutters::Vector{Segment}, bounding_box::NTuple{4, Float64}, box_size::Float64, center_point::Vector{Float64}}
const SensorSettings = @NamedTuple{fov::Float64, sensor_count::Int64, box_size::Float64, s_noise::Float64}

WGLMakie.@recipe(WorldMap) do scene
    WGLMakie.Attributes(
        labelworld = false,
        upcolor = :green,
    )
end

function WGLMakie.plot!(wm::WorldMap{<:Tuple{World}})
    world = wm[1]
    # labelworld = wm.labelworld
    walls = world[].walls
    for w in walls
        WGLMakie.lines!(wm, stack([first(w), last(w)]), color=:black)
    end
    wm
end

function WGLMakie.convert_arguments(::Type{<:WGLMakie.Arrows}, poses::Vector{Pose})
    x = WGLMakie.Point2f.(position.(poses))
    theta = WGLMakie.Point2f.(orientation.(poses))
    return (x,theta)
end

function WGLMakie.convert_arguments(::Type{<:WGLMakie.Arrows}, pose::Pose)
    x = WGLMakie.Point2f(position(pose))
    theta = WGLMakie.Point2f(orientation(pose))
    return ([x], [theta])
end

function WGLMakie.convert_arguments(::Type{<:WGLMakie.Scatter}, pose::Pose)
    return (WGLMakie.Point2f(position(pose)),)
end

function WGLMakie.convert_arguments(::Type{<:WGLMakie.LineSegments}, pose::Pose, sensor_settings::SensorSettings, readings::AbstractVector{Float64})
    x = position(pose)
    projections = [step_along_pose(rotate_pose(pose, sensor_angle(sensor_settings, j)), s) for (j, s) in enumerate(readings)]
    return ([1],[1])
end

function plot_sensors!(pose, color, readings, label, sensor_settings)
    plot!([position(pose)[1]], [position(pose)[2]]; color=color, label=nothing, seriestype=:scatter, markersize=3, markerstrokewidth=0)
    projections = [step_along_pose(rotate_pose(pose, sensor_angle(sensor_settings, j)), s) for (j, s) in enumerate(readings)]
    plot!(first.(projections), last.(projections);
            color=:blue, label=label, seriestype=:scatter, markersize=3, markerstrokewidth=1, alpha=0.25)
    plot!([Segment(position(pose), pr) for pr in projections]; color=:blue, label=nothing, alpha=0.25)
end

function frame_from_sensors(world, title, poses, poses_color, poses_label, pose, readings, readings_label, sensor_settings; show_clutters=false)
    the_plot = plot_world(world, title; show_clutters=show_clutters)
    plot!(poses; color=poses_color, label=poses_label)
    plot_sensors!(pose, poses_color, readings, readings_label, sensor_settings)
    return the_plot
end;

In [10]:
world, robot_init, robot_inputs, T = load_world("../example_20_program.json");

world_inputs = (walls = world.walls, bounce = 0.1)

path_integrated = integrate_controls(robot_init, robot_inputs, world_inputs)


20-element Vector{Pose}:
 Pose([1.8437380952380948, 16.669857142857147] 0.08090409915523009)
 Pose([1.2180983085992596, 16.619129592589132] -0.5467888408892483)
 Pose([1.2632405428236255, 16.591651710887344] -0.49394136891957907)
 Pose([1.6981416432133127, 16.357474195292898] 0.0)
 Pose([1.0402530380311035, 16.357474195292898] -0.6578886051822093)
 Pose([0.46019822709756464, 16.805698367377904] -1.390942827002419)
 Pose([0.4280250549753868, 16.982650814049883] -1.5707963267948966)
 Pose([0.4280250549753868, 16.771557480827138] -1.3597029935721499)
 Pose([0.4577566155201923, 16.632810198284712] -1.2178059389679858)
 Pose([0.7422772489355456, 15.860539907585897] -0.3947911196997622)
 Pose([0.8271137886199686, 15.825191349384054] -0.30288486837497053)
 Pose([1.082137242847622, 15.745496519937912] -0.03569911267932424)
 Pose([1.0793955135004187, 15.74559443884317] -0.038442590021188286)
 Pose([1.1178097011608614, 15.744116970087] -1.3877787807814457e-17)
 Pose([1.3152052610107436, 15.74411

In [11]:
WGLMakie.activate!(inline=true)

In [32]:

# %% [markdown]
# Following this initial display of the given data, we *suppress the clutters* until much later in the notebook.

f = Figure()
ax = Axis(f[1,1], aspect=DataAspect(), title="Start Pose Prior")
worldmap!(f[1,1], world)
gridbottom = GridLayout(f[2,1])
sl_steps = Makie.Slider(gridbottom[1,2], range=1:20)
N_particles = sl_steps.value
n_particle_tex = lift(N->"# steps $(N[])", N_particles)
Label(gridbottom[1,1], n_particle_tex)
displayed_path = lift((N,path)->path[1:N], N_particles, path_integrated) 
arrows!(f[1,1], displayed_path)
f

# %% [markdown]
# We can also visualize the behavior of the model of physical motion:
#
# ![](imgs_stable/physical_motion.gif)

# %% [markdown]
# ## Gen basics
#
# %% [markdown]
# ### Components of the motion model
#
# We start with the two building blocks: the starting pose and individual steps of motion.

# %%

In [38]:
@gen (static) function start_pose_prior(start, motion_settings)
    p ~ mvnormal(position(start), motion_settings.p_noise^2 * [1 0 ; 0 1])
    hd ~ normal(angle(start), motion_settings.hd_noise)
    return Pose(p, hd)
end

@gen (static) function step_model(start, control, world_inputs, motion_settings)
    position_new = step_along_pose(start, control.ds)
    p ~ mvnormal(position_new, motion_settings.p_noise^2 * [1 0 ; 0 1])
    hd ~ normal(angle(start) + control.dhd, motion_settings.hd_noise)
    return physical_step(position(start), p, hd, world_inputs)
end

# %%
motion_settings = (p_noise = 0.5, hd_noise = 2π / 360)

N_samples = 50
pose_samples = [start_pose_prior(robot_init, motion_settings) for _ in 1:N_samples]

std_devs_radius = 2.5 * motion_settings.p_noise

# Map visualization
f = Figure()
ax = Axis(f[1,1], aspect=DataAspect(), title="Start Pose Prior")

worldmap!(f[1,1], world)

# Sliders for motion settings
gridbottom = GridLayout(f[2,1])
p_noise_slider = Makie.Slider(gridbottom[1,3], range=range(0.01, 1.0, 100))
hd_noise_slider = Makie.Slider(gridbottom[2,3], range=range(0.01, 1.0, 100))
motion_settings = Makie.lift(p_noise_slider.value, hd_noise_slider.value) do p_noise, hd_noise
    (p_noise = p_noise, hd_noise=hd_noise)
end

pose_samples = lift(motion_settings) do motion_settings
    [start_pose_prior(robot_init, motion_settings) for _ in 1:20]
end
# Sliders for samples
reset_n_particles_btn = Makie.Button(gridbottom[3,1], label="Resample")
n_particle_slider = Makie.Slider(gridbottom[3,3], range=1:20, startvalue=5)
n_particles_obs = n_particle_slider.value

on(reset_n_particles_btn.clicks) do n
    reset_n_particles_btn.clicks
    pose_samples[] = [start_pose_prior(robot_init, motion_settings[]) for _ in 1:N_samples]
    notify(pose_samples)
end

# Numerical labels
p_noise_tex = lift(p_noise->"σ_p = $(p_noise[])", p_noise_slider.value)
hd_noise_tex = lift(hd_noise->"σ_hd = $(hd_noise[])", hd_noise_slider.value)
n_particle_tex = lift(N->"# of samples $(N[])", n_particles_obs)

Label(gridbottom[1,2], p_noise_tex)
Label(gridbottom[2,2], hd_noise_tex)
Label(gridbottom[3,2], n_particle_tex)
displayed_path = lift((N,path)->path[1:N], n_particles_obs, pose_samples) 
arrows!(f[1,1], displayed_path)
# Figure out how to get the correct units
scatter!(f[1,1], position(robot_init)..., markersize=std_devs_radius*p_noise_slider.value[], color = Makie.RGBA(1.0, .121, .121, 0.2))
f


In [ ]:
# %%
N_samples = 50
motion_settings = (p_noise = 0.5, hd_noise = 2π / 360)
noiseless_step = step_along_pose(robot_init, robot_inputs[1].ds)
step_samples = [step_model(robot_init, robot_inputs[1], world_inputs, motion_settings) for _ in 1:N_samples]

# Map Visualization
f = GLMakie.Figure()
ax = GLMakie.Axis(f[1,1], aspect=GLMakie.DataAspect(), title="Motion step model (samples)")
worldmap!(f[1,1], world)

# Sliders
gridbottom = GLMakie.GridLayout(f[2,1])
ds_slider = GLMakie.Slider(gridbottom[1,2], range=0:0.1:3.0, startvalue=1.6)
dhd_slider = GLMakie.Slider(gridbottom[2,2], range=range(0, 2π, 100))
p_noise_slider = GLMakie.Slider(gridbottom[3,2], range=range(0.01, 1.0, 100), startvalue=.3)
hd_noise_slider = GLMakie.Slider(gridbottom[4,2], range=range(0.01, 1.0, 100), startvalue=0.3)
n_particle_slider = GLMakie.Slider(gridbottom[5,2], range=1:20, startvalue=4)
# reset_n_particles_btn = GLMakie.Button(f[3,1], label="Resample")

# Slider Text
ds_tex = GLMakie.lift(ds->@sprintf("ds = %.3f", ds[]), ds_slider.value)
dhd_tex = GLMakie.lift(dhd->@sprintf("dhd = %.3f", dhd[]), dhd_slider.value)
p_noise_tex = GLMakie.lift(p_noise->@sprintf("σ_p = %.3f", p_noise[]), p_noise_slider.value)
hd_noise_tex = GLMakie.lift(hd_noise->@sprintf("σ_hd = %.3f", hd_noise[]), hd_noise_slider.value)
n_particle_tex = GLMakie.lift(N->"samples = $(N[])", n_particle_slider.value)

GLMakie.Label(gridbottom[1,1], ds_tex)
GLMakie.Label(gridbottom[2,1], dhd_tex)
GLMakie.Label(gridbottom[3,1], p_noise_tex)
GLMakie.Label(gridbottom[4,1], hd_noise_tex)
GLMakie.Label(gridbottom[5,1], n_particle_tex)

# Model Makie Observables
next_control = GLMakie.lift((ds, dhd)-> Control(ds,dhd), ds_slider.value, dhd_slider.value)

motion_settings = GLMakie.lift(
    (p_noise, hd_noise)-> (p_noise = p_noise, hd_noise=hd_noise),
    p_noise_slider.value, 
    hd_noise_slider.value
    )
n_particles = n_particle_slider.value

step_samples = GLMakie.lift(next_control, motion_settings) do control, motion_settings
    [step_model(robot_init, control, world_inputs, motion_settings) for _ in 1:N_samples]
end

noiseless_step = GLMakie.lift(next_control) do control
    n = step_along_pose(robot_init, control.ds)
    GLMakie.Point2f(n)
end
# GLMakie.on(reset_n_particles_btn.clicks) do n
#     reset_n_particles_btn.clicks
#     pose_samples[] = [start_pose_prior(robot_init, motion_settings[]) for _ in 1:N_samples]
#     GLMakie.notify(pose_samples)
# end


# Visual Makie Observables
step_samples_visible = GLMakie.lift(n_particle_slider.value, step_samples) do N, data
    return data[1:N]
end

GLMakie.scatter!(f[1,1], noiseless_step)
GLMakie.arrows!(f[1,1], robot_init, color=:blue)
GLMakie.arrows!(f[1,1], step_samples_visible, color=:red)
f

# %% [markdown]
# ### Noisy sensors
#
# We assume that the sensor readings are themselves uncertain, say, the distances only knowable up to some noise.  We model this as follows.  (We satisfy ourselves with writing a loop in the dynamic DSL because we will have no need for incremental recomputation within this model.)

# %%
"""
    sensor_model(pose::Pose, walls, sensor_settings)

Simulates the sensor model of the robot at `pose`, returning distance readings from the `walls` to sensors on the robot.

The `sensor_settings` controls the FoV, number of sensors, range of sensors, and the reading noise.
"""
@gen function sensor_model(pose::Pose, walls, sensor_settings)
    for j in 1:sensor_settings.sensor_count
        sensor_pose = rotate_pose(pose, sensor_angle(sensor_settings, j))
        {j => :distance} ~ normal(sensor_distance(sensor_pose, walls, sensor_settings.box_size), sensor_settings.s_noise)
    end
end

function noisy_sensor(pose, walls, sensor_settings)
    trace = simulate(sensor_model, (pose, walls, sensor_settings))
    return [trace[j => :distance] for j in 1:sensor_settings.sensor_count]
end

include("plotting.jl")

f = GLMakie.Figure()
ax = GLMakie.Axis(f[1,1], aspect=GLMakie.DataAspect(), title="Motion step model (samples)")
# worldmap!(f[1,1], world)


readings = [trace[j => :distance] for j in 1:sensor_settings.sensor_count]
GLMakie.linesegments!(f[1,1], robot_init, sensor_settings)
frame_plot = frame_from_sensors_trace(
    world, "Sensor model (samples)",
    path_integrated, :green2, "some path",
    pose, trace
    )
f





# %% [markdown]
# ### Full model
#
# We fold the sensor model into the motion model to form a "full model", whose traces describe simulations of the entire robot situation as we have described it.

# %%
prefix_address(t, rest) = (t == 1) ? (:initial => rest) : (:steps => (t-1) => rest)
get_path(trace) = [trace[prefix_address(t, :pose)] for t in 1:(get_args(trace)[1]+1)];

@gen (static) function full_model_initial(robot_init, walls, full_settings)
    pose ~ start_pose_prior(robot_init, full_settings.motion_settings)
    {:sensor} ~ sensor_model(pose, walls, full_settings.sensor_settings)
    return pose
end

@gen (static) function full_model_kernel(t, state, robot_inputs, world_inputs, full_settings)
    pose ~ step_model(state, robot_inputs[t], world_inputs, full_settings.motion_settings)
    {:sensor} ~ sensor_model(pose, world_inputs.walls, full_settings.sensor_settings)
    return pose
end
full_model_chain = Unfold(full_model_kernel)

@gen (static) function full_model(T, robot_init, robot_inputs, world_inputs, full_settings)
    initial ~ full_model_initial(robot_init, world_inputs.walls, full_settings)
    steps ~ full_model_chain(T, initial, robot_inputs, world_inputs, full_settings)
end

function sensor_distance(pose, walls, box_size)
    d = minimum(distance(pose, seg) for seg in walls)
    # Capping to a finite value avoids issues below.
    return isinf(d) ? 2. * box_size : d
end;

sensor_angle(sensor_settings, j) =
    sensor_settings.fov * (j - (sensor_settings.sensor_count - 1) / 2) / (sensor_settings.sensor_count - 1)

motion_settings = (p_noise = 0.5, hd_noise = 2π / 360)
sensor_settings = (fov = 2π*(2/3), sensor_count = 41, box_size = world.box_size, s_noise=0.10)
robot_settings = (motion_settings=motion_settings, sensor_settings=sensor_settings)
full_model_args = (T, robot_init, robot_inputs, world_inputs, robot_settings)

trace = simulate(full_model, full_model_args);
selection = select((prefix_address(t, :pose) for t in 1:3)..., (prefix_address(t, :sensor => j) for t in 1:3, j in 1:5)...)
get_selected(get_choices(trace), selection)

# %% [markdown]
# In the math picture, `full_model` corresponds to a distribution $\text{full}$ over its traces.  Such a trace is identified with of a pair $(z_{0:T}, o_{0:T})$ where $z_{0:T} \sim \text{path}(\ldots)$ and $o_t \sim \text{sensor}(z_t, \ldots)$ for $t=0,\ldots,T$.  The density of this trace is then
# $$\begin{align*}
# P_\text{full}(z_{0:T}, o_{0:T})
# &= P_\text{path}(z_{0:T}) \cdot \prod\nolimits_{t=0}^T P_\text{sensor}(o_t) \\
# &= \big(P_\text{start}(z_0)\ P_\text{sensor}(o_0)\big)
#   \cdot \prod\nolimits_{t=1}^T \big(P_\text{step}(z_t)\ P_\text{sensor}(o_t)\big).
# \end{align*}$$
#
# By this point, visualization is essential.

# %%
function get_sensors(trace)
    display(trace)
    [[trace[prefix_address(t, :sensor => j => :distance)]
      for j in 1:get_args(trace)[5].sensor_settings.sensor_count]
     for t in 1:(get_args(trace)[1]+1)];
end

function frames_from_full_trace(world, title, trace; show_clutters=false)
    T = get_args(trace)[1]
    robot_init = get_args(trace)[2]
    robot_inputs = get_args(trace)[3]
    poses = get_path(trace)
    noiseless_steps = [position(robot_init), [step_along_pose(pose, c.ds) for (pose, c) in zip(poses, robot_inputs)]...]
    settings = get_args(trace)[5]
    std_devs_radius = 2.5 * settings.motion_settings.p_noise
    sensor_readings = get_sensors(trace)
    plots = Vector{Plots.Plot}(undef, 2*(T+1))
    for t in 1:(T+1)
        frame_plot = plot_world(world, title; show_clutters=show_clutters)
        plot!(poses[1:t-1]; color=:black, label="past poses")
        plot!(make_circle(noiseless_steps[t], std_devs_radius);
              color=:red, linecolor=:red, label="95% region", seriestype=:shape, alpha=0.25)
        plot!(Pose(trace[prefix_address(t, :pose => :p)], angle(poses[t])); color=:red, label="sampled next step")
        plots[2*t-1] = frame_plot
        plots[2*t] = frame_from_sensors(
            world, title,
            poses[1:t], :black, nothing,
            poses[t], sensor_readings[t], "sampled sensors",
            settings.sensor_settings; show_clutters=show_clutters)
    end
    return plots
end;

In [48]:
f = Figure()
f

In [49]:
f = Figure()
f

InterruptException: InterruptException: